In [1]:
# Imports and setup
import sys
import os
import json
sys.path.append("../utils")
from datetime import datetime
from gpt_utils import load_preferences, load_text, load_prompt, fill_prompt

In [2]:
from llm_wrapper import call_mistral
from chunker import summarize_large_text
from image_caption import generate_tagged_picture_insights
from input_handler import get_all_text, get_all_images
from image_to_text import get_dating_bio_summary
from image_to_text import get_linkedin_summary
from behavior_scoring import analyze_behavior
from time_scoring import analyze_timing
from image_to_text import extract_text_from_image

In [3]:
# Load preferences and pictures
prefs = load_preferences('../preferences.json')
pictures_folder = "../data/pictures/"

In [4]:
#LinkedIn summary (image if available, else txt)
linkedin = get_linkedin_summary("../data/linkedin/")
print (linkedin)

Experienced Software Engineer with a B.Tech in Computer Science from IIT Bombay. Currently working as a Senior Software Engineer at FinTechX, specializing in building scalable backend systems for payment platforms and NLP-powered fraud detection. Previously, worked at Flipkart where responsibilities included designing distributed systems for e-commerce search and optimizing infrastructure for 10M+ daily queries. Skilled in Python, Java, C++, Machine Learning, Natural Language Processing, Deep Learning, AWS, and GCP. Based in Bangalore, India.


In [5]:
# Dating bio summary (image if available, else txt)
bio = get_dating_bio_summary("../data/dating_app/")
print (bio)

The user is an adventurous fitness enthusiast with a passion for food who enjoys spontaneous road trips, deep conversations, and good coffee. They are seeking a meaningful connection beyond swipes, and share interests in mountains and pasta.


In [6]:
chat_folder = "../data/chats/"

# 1. Get all .txt chat content
chat_text = get_all_text(chat_folder)

# 2. Get all image captions
chat_images = get_all_images(chat_folder)
chat_img_texts = [extract_text_from_image(img) for img in chat_images]
chat_img_text = "\n".join(chat_img_texts)

# 3. Merge both
combined_chat = f"{chat_text}\n{chat_img_text}".strip()

# 4. Use this combined version for summarization and scoring
summarized_chats = summarize_large_text(combined_chat, call_mistral, max_tokens=1500)
time_scores = analyze_timing(combined_chat)
print("time_scores:", time_scores)
time_block = f"""
Ghosting Score: {time_scores.get('ghosting_score', 'NA')}
Engagement Decay: {time_scores.get('engagement_decay', '')}
Response Consistency: {time_scores.get('response_consistency', '')}
Comment: {time_scores.get('comment', '')}
"""

time_scores: {'ghosting_score': 0, 'engagement_decay': 'low', 'consistency_score': 8, 'quickness_score': 6, 'comment': "The engagement is relatively consistent with occasional quick replies. There's a slight decay in enthusiasm, but the overall interaction remains active."}


In [7]:

behavior_scores = analyze_behavior(summarized_chats)
print("Behavior Scores:", behavior_scores)

behavior_block = f"""
Behavioral Analysis:
- Tone: {behavior_scores.get('tone', 'NA')}
- Emotional Availability: {behavior_scores.get('emotional_availability', 'NA')}
- Wit: {behavior_scores.get('wit', 'NA')}
- Love Bombing: {behavior_scores.get('love_bombing', 'NA')}
- Comment: {behavior_scores.get('comment', '')}
""".strip()

Behavior Scores: {'tone': 8, 'emotional_availability': 7, 'wit': 6, 'love_bombing': 0, 'comment': "The conversation demonstrates a friendly and supportive tone with emotional clarity. Both individuals display maturity in their interactions and show empathy towards each other. While the chat includes occasional humor, it's not consistently witty. No evidence of love-bombing was found."}


In [8]:
# Notes (optional)
notes = load_text("../data/notes.txt")

In [9]:
# Profile pictures vibe tagging
pictures_folder = "../data/pictures/"
if os.path.exists(pictures_folder) and os.listdir(pictures_folder):
    picture_insight = generate_tagged_picture_insights(pictures_folder)
else:
    picture_insight = "No profile pictures provided."

In [10]:
# Fill the GPT prompt
template = load_prompt('../prompts/match_prompt.txt')
final_prompt = fill_prompt(template, prefs, linkedin, bio, summarized_chats, notes, picture_insight, behavior_block,time_block)
print(final_prompt)

You are a brutally honest compatibility evaluator. No sugarcoating.

Your job is to analyze the profile, pictures, and chats of a potential partner and return:

1. Red Flags
2. Compatibility Score (0–100%)
3. Match Breakdown: For each key preference, score the match on a scale of 0 (no match) to 10 (perfect match).  
If the relevant detail is **not available** in the guy's profile, simply **omit** the key from output.
4. Behavioral Analysis Summary (tone, wit, emotional availability)
5. Time-Based Engagement Summary (ghosting, consistency, energy drop)
6. Final Verdict (Proceed / Pause / No)

---

Preferences (Non-negotiables, Expectations):
{
  "preferences": {
    "personality_traits": [
      "tall",
      "emotionally available",
      "mature",
      "witty",
      "communicative",
      "consistent",
      "uses proper grammar"
    ],
    "intent": "long-term relationship or marriage",
    "income_min": 0,
    "food_preferences": "not important",
    "substance_use": {
      "smo

In [11]:
output = call_mistral(final_prompt)
print(" GPT Structured Response:\n", output)

 GPT Structured Response:
  {
     "red_flags": [],
     "compatibility_score": 80,
     "match_breakdown": {
       "tall": undefined, // Not mentioned in the profile
       "emotionally available": 7,
       "caste_match": undefined, // Not applicable
       "income_match": undefined, // Not mentioned in the profile
       "personality_traits": 6, // Somewhat matches as he is adventurous and communicative
       "intent": 10, // Both are seeking long-term relationships or marriage
       "food_preferences": undefined, // Not mentioned in the profile
       "substance_use": {
         "smoking": 0, // Not mentioned in the profile
         "alcohol": 0, // Not mentioned in the profile
         "drugs": 0 // Not mentioned in the profile
       },
       "picture_traits": {
         "explorer vibe": 10, // Has a mountain picture which showcases his explorer side
         "travel pictures": 8, // Has travel pictures but not as frequent as selfies
         "food pictures": 7, // Food pictu

In [13]:
# Save the output to markdown and log to JSON
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
md_filename = f"report_{timestamp}.md"
md_path = f"../reports/{md_filename}"

# Save markdown
with open(md_path, "w", encoding="utf-8") as f:
    f.write(output)
print(f"Report saved as: {md_path}")

# Save to history.json
log = {
    "timestamp": timestamp,
    "preferences": json.loads(prefs),
    "linkedin": linkedin,
    "bio": bio,
    "chat_summary": summarized_chats,
    "notes": notes,
    "picture_insight": picture_insight,
    "report_file": md_filename,
    "output": output
}

history_path = "../results/history.json"
os.makedirs("../results", exist_ok=True)

if os.path.exists(history_path):
    with open(history_path, "r", encoding="utf-8") as f:
        history = json.load(f)
else:
    history = []

history.append(log)

with open(history_path, "w", encoding="utf-8") as f:
    json.dump(history, f, indent=2)

print("Evaluation added to history.json")

Report saved as: ../reports/report_2025-09-14_17-24-00.md
Evaluation added to history.json


In [15]:
import re
import json

def extract_scores(output: str) -> dict:
    try:
        # Extract JSON block
        match = re.search(r'\{.*?\}', output, re.DOTALL)
        if not match:
            print(" No JSON found in GPT output.")
            return {}

        json_part = match.group(0)

        # Remove any inline comments (e.g. // or #)
        cleaned_json = re.sub(r'//.*?$|#.*?$', '', json_part, flags=re.MULTILINE)

        return json.loads(cleaned_json)
    except Exception as e:
        print(" Error parsing scores:", e)
        return {}

